In [15]:
import sys
sys.path.insert(0, "..")
import os
from dotenv import load_dotenv
from typing import Optional, List
from azure.core.credentials import AzureSasCredential
#from azure.core.exceptions import ResourceNotFoundError
from azure.storage.blob import ContentSettings, PublicAccess
from azure.storage.blob.aio import BlobServiceClient, ContainerClient
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from app.services.azure_store_service import AzureStoreService
from app.services.stt_service import SttService
from uuid import uuid4
import unittest
import json

from loguru import logger
load_dotenv("../.dev.env")

True

In [33]:
def generate_id():
    return str(uuid4())

In [34]:
user_id = generate_id()
user_id

'afb73f48-9008-4120-8918-52790c437fab'

In [35]:
account_url = os.getenv("AZURE_ACCOUNT_URL")
#connection_string = os.getenv("AZURE_STORAGE_CONNECTION_STRING")
user_id = generate_id()
container_name = "worklog-s2t"

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
chat_deployment = os.getenv("AZURE_GPT_DEPLOYMENT_NAME")
      
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2024-02-01",
)

In [36]:
speech_deployment = os.getenv("AZURE_SPEECH_DEPLOYMENT_NAME")
audio_test_file = "../app/data/1_766414.wav"
    
result = client.audio.transcriptions.create(
    file=open(audio_test_file, "rb"),            
    model=speech_deployment
    )
    
print(result)

Transcription(text="De melding is van werkoorder 766414 is de deur defect, deur 3. Oké, fotocel zender is vervangen. De melding was fotocel deur 3B blijft geblokkeerd staan. Dat is gelijk niet zo'n heel duidelijk logboek. Lijst, pakken we een andere.")


In [38]:
result.text

"De melding is van werkoorder 766414 is de deur defect, deur 3. Oké, fotocel zender is vervangen. De melding was fotocel deur 3B blijft geblokkeerd staan. Dat is gelijk niet zo'n heel duidelijk logboek. Lijst, pakken we een andere."

In [52]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
token_provider = get_bearer_token_provider(
            DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

langchain_llm = AzureChatOpenAI(
    model=os.environ["AZURE_GPT_DEPLOYMENT_NAME"],
    azure_endpoint=endpoint,
    azure_ad_token_provider=token_provider,
    api_version="2024-02-01",
    )

In [53]:
from pathlib import Path

human_prompt = Path('../app/prompts/storingstemplate.txt').read_text()

system_prompt = Path('../app/prompts/systeem_prompt.txt').read_text()

In [54]:
system_prompt

'Jij bent een Nederlandstalige AI-assistent die trambestuurders en monteurs helpt bij het versturen van meldingen of het stellen van vragen over storingen of reparaties.\nDe invoer in jouw systeem mag in verschillende talen zijn, maar jouw uitvoer moet in het Nederlands zijn.\nZowel in de invoer als in de uitvoer kunnen in de overigens Nederlandse tekst technische werkvoorwaarden in het Engels voorkomen.'

In [55]:
prompt_template = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                ("human", human_prompt),
            ]
        )
chain = prompt_template | langchain_llm
answer_object = chain.invoke(
        {
                "language": "nl",
                "transcript": result.text,
            }
        )

In [56]:
answer_object.content

'Bedankt voor de informatie, ik ga nu de gegeven tekst vertalen naar de gedefinieerde template: \n\n1. Specifiek onderdeel van het voertuig waar de storing optrad:\n    a. Bij een deurstoring, welke deur? Deur 3B.\n    \n2. Aanleiding- Korte omschrijving van de aanleiding voor de storing:\n    a. Bestuurder of monteur constateert een probleem en rapporteert deze. De melding was fotocel deur 3B blijft geblokkeerd staan.\n\n3. Type Storing: Korte omschrijving van het type storing. Deurstoring - defecte fotocel.\n\n4. Gedetailleerde Omschrijving: Deur 3B had een defecte fotocel die ervoor zorgde dat de deur geblokkeerd bleef staan.\n\n5. Ondernomen Acties: De fotocel zender van deur 3B is vervangen om het probleem te verhelpen.\n\n6. Verbruikte Materialen: **Niet gespecificeerd in de geleverde tekst, maar vermoedelijk is er een nieuwe fotocel zender gebruikt.**\n\n7. Resultaat: **Niet gespecificeerd in de geleverde tekst, het is niet duidelijk of de storing volledig is opgelost na het ver